corr 0.8 미만의 column으로 진행

In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [40]:
train = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_case3_8.csv')
train_problem = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_train_problem.csv', parse_dates=['time'])
test = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\test_case3_8.csv')
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

# scaling

## train

In [41]:
# multiple_model 만 범주형 데이터

categorical_df = train[['multiple_model']]
numerical_df = train.drop(columns=['multiple_model'])
numerical_df

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period
0,316.0,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
1,2345.0,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,30.0
2,305.0,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
3,297.0,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,30.0
4,777.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,191.0,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,NaN
14996,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
14997,818.0,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,12.0,0.0,1.0,0.0,0.0,58.0,7.0,5.0,NaN
14998,144.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,2.0,0.0,1.0,0.0,0.0,6.0,0.0,0.0,NaN


In [42]:
# 수치형 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scalied_df = numerical_df.copy()

scalied_df = scaler.fit_transform(scalied_df)
numerical_df = pd.DataFrame(scalied_df, columns=numerical_df.columns)
numerical_df.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.444763
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.444763
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.405319
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.444763
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.364376,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.444763


In [43]:
train = numerical_df.join(categorical_df)
train.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.444763,0.0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.444763,0.0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.405319,0.0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.444763,0.0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.444763,0.0


## test

In [44]:
# multiple_model 만 범주형 데이터

categorical_df = test[['multiple_model']]
numerical_df = test.drop(columns=['multiple_model'])
numerical_df

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period
0,2731,0.0,2669.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,1.0,3.0,0.0,2.0,92.0,0.0,2.0,29
1,284,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
2,903,903.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,0.0,14.0,113.0,1.0,2.0,30
3,328,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,7.0,3.0,1.0,0.0,0.0,40.0,8.0,0.0,28
4,871,0.0,622.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,5.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,1115,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,30
14994,515,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,1.0,0.0,1.0,0.0,0.0,12.0,17.0,0.0,30
14995,1915,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,802.0,0.0,1.0,1.0,0.0,329.0,2.0,3.0,30
14996,24669,24669.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,0.0,1.0,3.0,0.0,30.0,49.0,0.0,28


In [45]:
scalied_df = numerical_df.copy()

scalied_df = scaler.fit_transform(scalied_df)
numerical_df = pd.DataFrame(scalied_df, columns=numerical_df.columns)
numerical_df

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period
0,0.432273,-0.282718,1.465372,-0.138438,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.347687,-0.052885,0.017059,3.792035,-0.133970,0.040193,0.539686,-0.390797,-0.080517,0.269713
1,-0.191518,-0.282718,-0.227667,-0.190118,0.051198,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-1.000390,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,-0.602594,-0.390797,-0.645212,0.095698
2,-0.033722,0.638161,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.347687,-0.052885,-0.075341,0.593961,-0.133970,0.801722,0.800423,-0.334183,-0.080517,0.443729
3,-0.180302,0.051777,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.663371,-0.004461,0.201860,0.593961,-0.133970,-0.086729,-0.105951,0.062114,-0.645212,0.095698
4,-0.041880,-0.282718,0.166889,0.017435,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.073587,-0.018296,0.017059,0.593961,0.288682,-0.086729,-0.006622,4.704445,-0.645212,0.443729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,0.020321,-0.282718,0.479616,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.347687,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,0.639014,4.534603,-0.645212,0.443729
14994,-0.132632,0.242479,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.179178,-0.045967,-0.075341,0.593961,-0.133970,-0.086729,-0.453601,0.571638,-0.645212,0.443729
14995,0.224257,1.670199,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.010668,5.495079,-0.075341,0.593961,0.288682,-0.086729,3.482297,-0.277569,0.201830,0.443729
14996,6.024724,24.874719,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.179178,-0.052885,-0.075341,0.593961,1.133985,-0.086729,-0.230111,2.383279,-0.645212,0.095698


In [46]:
test = numerical_df.join(categorical_df)
test.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model
0,0.432273,-0.282718,1.465372,-0.138438,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,0.017059,3.792035,-0.133970,0.040193,0.539686,-0.390797,-0.080517,0.269713,1
1,-0.191518,-0.282718,-0.227667,-0.190118,0.051198,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,-0.602594,-0.390797,-0.645212,0.095698,0
2,-0.033722,0.638161,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,-0.075341,0.593961,-0.133970,0.801722,0.800423,-0.334183,-0.080517,0.443729,0
3,-0.180302,0.051777,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.004461,0.201860,0.593961,-0.133970,-0.086729,-0.105951,0.062114,-0.645212,0.095698,0
4,-0.041880,-0.282718,0.166889,0.017435,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.018296,0.017059,0.593961,0.288682,-0.086729,-0.006622,4.704445,-0.645212,0.443729,1


In [47]:
train.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.444763,0.0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.444763,0.0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.405319,0.0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.444763,0.0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.444763,0.0


# learning

In [48]:
train = train.to_numpy()
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1

train_x = train
train_y = problem

In [49]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001771 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4699
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.793829	valid_0's pr_auc: 0.798678
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000964 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4860
[LightGBM] [Info] Numbe

In [50]:
print(np.mean(auc_scores))

0.7978115502603672


In [51]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,problem
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.444763,0.0,0.0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.444763,0.0,1.0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.405319,0.0,0.0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.444763,0.0,0.0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.444763,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.316156,-0.340504,-0.239957,-0.053688,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.491089,0.019092,0.482907,NaN,0.0,0.0
14996,-0.387084,-0.340504,-0.239957,-0.414143,-0.062749,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,NaN,0.0,0.0
14997,-0.078337,0.704060,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,0.115262,0.019092,0.764726,NaN,0.0,1.0
14998,-0.333983,-0.156620,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.074159,0.628662,-0.114398,-0.080131,-0.528213,-0.336841,-0.644371,NaN,0.0,1.0


In [56]:
clf = setup(data = train, target = "problem", silent=True, train_size=0.85) 

,Description,Value
0,session_id,2679
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 64)"
5,Missing Values,True
6,Numeric Features,57
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [57]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7919,0.8117,0.5136,0.7901,0.6223,0.4875,0.5091,0.0720
gbc,Gradient Boosting Classifier,0.7914,0.8107,0.4777,0.8236,0.6045,0.4762,0.5088,0.3340
rf,Random Forest Classifier,0.7899,0.8046,0.4925,0.8015,0.6100,0.4772,0.5038,0.1960
et,Extra Trees Classifier,0.7865,0.7997,0.4840,0.7967,0.6021,0.4678,0.4950,0.2090
ada,Ada Boost Classifier,0.7843,0.7965,0.4847,0.7879,0.6000,0.4636,0.4894,0.0920
lr,Logistic Regression,0.7752,0.7688,0.4206,0.8177,0.5551,0.4248,0.4670,0.1130
lda,Linear Discriminant Analysis,0.7635,0.7546,0.3550,0.8488,0.5004,0.3780,0.4398,0.0380
nb,Naive Bayes,0.6986,0.7365,0.4895,0.6424,0.5194,0.3192,0.3419,0.0160
knn,K Neighbors Classifier,0.7480,0.7259,0.4384,0.6942,0.5372,0.3760,0.3949,0.3180
dt,Decision Tree Classifier,0.6917,0.6556,0.5472,0.5380,0.5424,0.3100,0.3101,0.0340


In [58]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7859,0.8210,0.4930,0.7865,0.6061,0.4695,0.4936
1,0.7898,0.8045,0.4771,0.8169,0.6024,0.4726,0.5042
2,0.7886,0.8062,0.4783,0.8108,0.6016,0.4705,0.5009
3,0.7929,0.8158,0.4935,0.8124,0.6140,0.4838,0.5119
4,0.7972,0.8129,0.5088,0.8139,0.6262,0.4970,0.5228
Mean,0.7909,0.8121,0.4901,0.8081,0.6101,0.4787,0.5067
Std,0.0039,0.0061,0.0117,0.0110,0.0092,0.0105,0.0100


In [59]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7921,0.8210,0.4825,0.8122,0.6054,0.4764,0.5062


In [60]:
final_model = finalize_model(blended)

In [61]:
# pycaret의 predict_model을 사용하기 위해 test의 모든 컬럼을 정수로 변경
t_columns = list(np.arange(0,63))
test.columns = t_columns

In [62]:
prep_pipe = get_config('prep_pipe')
transformed_unseen_data = prep_pipe.transform(test)

In [29]:
transformed_unseen_data

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,62,10_0.0,15_0.0,23_0.0,24_18.0,25_0.0,61_0.0
0,2731.0,0.0,2669.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,92.0,0.0,2.0,29.0,1.0,1.0,1.0,0.0,1.0,0.0
1,284.0,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,28.0,1.0,1.0,1.0,0.0,1.0,0.0
2,903.0,903.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,113.0,1.0,2.0,30.0,1.0,1.0,1.0,0.0,1.0,0.0
3,328.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,40.0,8.0,0.0,28.0,1.0,1.0,1.0,0.0,1.0,0.0
4,871.0,0.0,622.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,90.0,0.0,30.0,1.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,1115.0,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,87.0,0.0,30.0,1.0,1.0,1.0,0.0,1.0,0.0
14994,515.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,17.0,0.0,30.0,1.0,1.0,1.0,0.0,1.0,0.0
14995,1915.0,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,329.0,2.0,3.0,30.0,1.0,1.0,1.0,0.0,1.0,0.0
14996,24669.0,24669.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,49.0,0.0,28.0,1.0,1.0,1.0,0.0,1.0,0.0


In [63]:
prections = final_model.predict_proba(transformed_unseen_data)[:,1]
prections

array([0.91368972, 0.689909  , 0.83302197, ..., 0.8107812 , 0.85298234,
       0.77252114])

In [64]:
prections.mean()

0.7855052470795081